# Import

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torch.optim as optim  # This is the critical import
from PIL import Image      # Should show 9.x.x or later
from torchsummary import summary
# import os
# import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


In [22]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, num_convs):
        super().__init__()
        self.convs = nn.Sequential()
        
        # Membuat layer konvolusi sesuai jumlah yang ditentukan
        for i in range(num_convs):
            input_channels = in_channels if i == 0 else out_channels
            self.convs.add_module(f'conv{i+1}', nn.Conv2d(
                input_channels, out_channels, kernel_size=3, padding=1))
            self.convs.add_module(f'bn{i+1}', nn.BatchNorm2d(out_channels))
            self.convs.add_module(f'relu{i+1}', nn.ReLU(inplace=True))
        
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Skip connection dengan 1x1 conv untuk menyesuaikan dimensi
        self.skip = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        identity = self.skip(x)  # Proses skip connection
        out = self.convs(x)      # Jalur utama melalui konvolusi
        out = self.pool(out)     # Pooling setelah konvolusi
        out += identity          # Tambahkan skip connection
        return F.relu(out)       # Aktivasi akhir

class ResidualVGG16(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        
        # Membangun blok-blok residual sesuai arsitektur VGG16
        self.blocks = nn.Sequential(
            ResidualBlock(3, 32, 2),       # Blok 1: 2 konvolusi 64 channel
            ResidualBlock(32, 64, 2),     # Blok 2: 2 konvolusi 128 channel
            ResidualBlock(64, 128, 3),    # Blok 3: 3 konvolusi 256 channel
            ResidualBlock(128, 256, 3),    # Blok 4: 3 konvolusi 512 channel
            ResidualBlock(256, 256, 3),    # Blok 5: 3 konvolusi 512 channel
        )
        
        # Classifier dengan fully-connected layers
        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.blocks(x)               # Loloskan melalui semua blok residual
        x = torch.flatten(x, 1)          # Flatten feature maps
        x = self.classifier(x)           # Loloskan melalui classifier
        return x



In [23]:

model = ResidualVGG16()  # Your model
summary(model, input_size=(3, 224, 224))  

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             128
       BatchNorm2d-2         [-1, 32, 112, 112]              64
            Conv2d-3         [-1, 32, 224, 224]             896
       BatchNorm2d-4         [-1, 32, 224, 224]              64
              ReLU-5         [-1, 32, 224, 224]               0
            Conv2d-6         [-1, 32, 224, 224]           9,248
       BatchNorm2d-7         [-1, 32, 224, 224]              64
              ReLU-8         [-1, 32, 224, 224]               0
         MaxPool2d-9         [-1, 32, 112, 112]               0
    ResidualBlock-10         [-1, 32, 112, 112]               0
           Conv2d-11           [-1, 64, 56, 56]           2,112
      BatchNorm2d-12           [-1, 64, 56, 56]             128
           Conv2d-13         [-1, 64, 112, 112]          18,496
      BatchNorm2d-14         [-1, 64, 1

In [24]:
# Contoh penggunaan
# if __name__ == "__main__":
#     model = ResidualVGG16(num_classes=2)
#     input_tensor = torch.randn(1, 3, 224, 224)  # Contoh input
#     output = model(input_tensor)
#     print(f"Output shape: {output.shape}")   

# Load Data

In [25]:
# tumor_dir_path = "./Dataset/processed/yes"
# no_tumor_dir_path = "./Dataset/processed/no"

# tumor_image_paths =  os.listdir(tumor_dir_path)
# tumor_image_paths = [os.path.join(tumor_dir_path, path) for path in tumor_image_paths]

# no_tumor_image_paths =  os.listdir(no_tumor_dir_path)
# no_tumor_image_paths = [os.path.join(no_tumor_dir_path, path) for path in no_tumor_image_paths]
# no_tumor_image_paths

# display(tumor_image_paths, no_tumor_image_paths)

In [26]:
# dataset_dir_path = '/kaggle/input/brain-tumor-dataset'
dataset_dir_path = './Dataset/final'

In [ ]:
# Update the data preparation function
def prepare_data(data_dir='your_dataset'):
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], 
        #                      std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406],
        #                      std=[0.229, 0.224, 0.225])
    ])

    # Load datasets
    train_dataset = datasets.ImageFolder(
        root=f'{data_dir}/train',
        transform=train_transform
    )
    
    val_dataset = datasets.ImageFolder(
        root=f'{data_dir}/val',
        transform=val_transform
    )

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)
    
    return train_loader, val_loader

In [28]:
train_loader, val_loader = prepare_data(data_dir=dataset_dir_path)

In [ ]:
# Hyperparameters
NUM_EPOCHS = 10


# Initialize model, loss, optimizer
model = ResidualVGG16(num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()  
    running_loss = 0.0
    total_batches = len(train_loader)
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        # Move data to device (if using GPU)
        images = images
        labels = labels
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Print statistics
        running_loss += loss.item()
        
        # Print every 10 batches
        if (batch_idx + 1) % 10 == 0:
            current_loss = running_loss / 10
            print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], '
                  f'Batch [{batch_idx+1}/{total_batches}], '
                  f'Loss: {current_loss:.4f}')
            running_loss = 0.0
    
    # Print epoch statistics
    epoch_loss = running_loss / len(train_loader)
    print(f'\nEpoch [{epoch+1}/{NUM_EPOCHS}], '
          f'Average Loss: {epoch_loss:.4f}\n')

KeyboardInterrupt: 

In [ ]:
import torch
print(torch.__version__)

2.7.0+cpu
